In [ ]:
"""総合演習2 マクロ系シミュレーション 最終課題3

1次元非定常拡散方程式のシミュレーション(有限差分法)
"""

import sys
from typing import Final, Self

import matplotlib.pyplot as plt
import numpy as np
from IPython.display import HTML
from matplotlib import animation

# ========== パラメータ ==========
LX: Final[float] = 10              # 部屋のx方向の長さ(m)
D: Final[float] = 10**(-2)         # 二酸化炭素の拡散係数(m^2/s)
C_OUT: Final[float] = 400          # 外気の二酸化炭素濃度(ppm)

NX: Final[int] = 1001              # x方向の格子点数
DT: Final[float] = 1               # 時間刻み幅(s)
T_END: Final[float] = 1000         # シミュレーション時間(s)
DT_PLOT: Final[float] = T_END/50   # グラフを作成する時間間隔(s)
# ==============================

# float型のNumPy配列の型エイリアス
FloatArray = np.typing.NDArray[np.float64]


def func_src(x: float, lx: float) -> float:
    """xにおける二酸化炭素の生成項を計算する

    Parameters
    ----------
    x : float
        x座標
    lx : float
        部屋のx方向の長さ

    Returns
    -------
    float
        xにおける二酸化炭素の生成項
    """

    return 1


class Variable:
    """格子点上の物理量を扱うクラス

    Attributes
    ----------
    value : FloatArray
        格子点上の物理量の値
    __name : str
        'grid_x', 'co2', 'src', 'lambda' のいずれか

    Notes
    -----
    インスタンスを生成する前に, set_class_variablesクラスメソッドを実行する必要あり
    Variableクラスの外で使用しない属性は, 変数名の最初にダンダー(__)をつけて隠蔽している
    """

    __lx: float
    __nx: int
    __dx: float

    # set_class_variablesクラスメソッドを実行したかどうかを判定する真偽値
    __flag: bool = False

    @classmethod
    def set_class_variables(cls, lx: float, nx: int) -> None:
        """Variableクラスのクラス変数を設定する

        Parameters
        ----------
        lx : float
            部屋のx方向の長さ
        nx : int
            x方向の格子点数
        """

        cls.__lx = lx
        cls.__nx = nx
        cls.__dx = lx / (nx-1)
        cls.__flag = True

    def __init__(self, name: str) -> None:
        """Variableクラスのイニシャライザ

        Parameters
        ----------
        name : str
            'grid_x', 'co2', 'src' のいずれか
        """

        if not Variable.__flag:
            print('[ERROR] set_class_variablesクラスメソッドが実行されていません')
            sys.exit()

        if name not in ['grid_x', 'co2', 'src']:
            print('[ERROR] Variableクラスのイニシャライザの引数が不適切です')
            sys.exit()
        self.__name: str = name

        nx: int = Variable.__nx
        self.value: FloatArray = np.empty(nx, dtype=np.float64)

        lx: float = Variable.__lx
        dx: float = Variable.__dx
        x: float
        for ix in range(nx):
            x = dx * ix
            self.value[ix] = self.set_initial_condition(x, lx)


    def set_initial_condition(self, x: float, lx: float) -> float:
        """xにおける(初期条件の)値を設定する

        Parameters
        ----------
        x : float
            x座標
        lx : float
            部屋のx方向の長さ

        Returns
        -------
        float
            xにおける(初期条件の)値
        """

        if self.__name == 'grid_x':
            return x
        elif self.__name == 'co2':
            return 5000
        elif self.__name == 'src':
            return func_src(x, lx)
        elif self.__name == 'lambda':
            return 0

    def set_boundary_condition(self, d: float, dt: float, src: Self, c_out: float) -> None:
        """境界条件を設定する

        Parameters
        ----------
        d : float
            二酸化炭素の拡散係数
        dt : float
            時間刻み幅
        src : Self | None, optional, default None
            Variableクラスのインスタンス(src)
        c_out : float | None, optional, default None
            外気の二酸化炭素濃度
        """

        if self.__name == 'co2':

            dx: float = Variable.__dx

            src_name: str = ''
            if isinstance(src, Variable):
                src_name = src.__name
            if (src_name != 'src'):
                print('[ERROR] co2.set_boundary_conditionメソッドの引数が不適切です')
                sys.exit()
            if c_out is None:
                print('[ERROR] co2.set_boundary_conditionメソッドの引数が不適切です')
                sys.exit()

            # 左端
            self.value[0] = c_out

            # 右端
            self.value[-1] += (
                2 * d * (self.value[-2] - self.value[-1]) / (dx**2) + src.value[-1]
            ) * dt

    def laplacian(self) -> FloatArray:
        """ラプラシアンを計算する

        Returns
        -------
        laplacian : FloatArray
            ラプラシアン
        """

        nx: int = Variable.__nx
        laplacian: FloatArray = np.zeros(nx, dtype=np.float64)

        dx: float = Variable.__dx
        for ix in range(1, nx-1):
            laplacian[ix] = (
                (self.value[ix+1] - 2 * self.value[ix] + self.value[ix-1]) / (dx**2)
            )
        return laplacian


def create_plot(frame: int, *fargs) -> None:
    """各タイムステップでのグラフを作成する
    
    animation.FuncAnimationに使用する関数
    
    Parameters
    ----------
    frame : int
        フレーム番号
    fargs : tuple
        格子点のx座標, 時刻, co2の結果, 測定点, 測定データを格納したタプル
    """

    grid_x: FloatArray
    results_t: list[float]
    results_co2: list[FloatArray]
    grid_x, results_t, results_co2 = fargs
    
    plt.cla()

    # co2のグラフ
    co2: FloatArray = results_co2[frame]
    plt.scatter(grid_x, co2, c='red', label='CO2')
    plt.ylim(400, 7000)
    axes.set_xlabel('x')
    plt.title(f't = {results_t[frame]:.1f} sec')
    plt.legend()


if __name__ == '__main__':

    Variable.set_class_variables(LX, NX)

    # Variableクラスのインスタンスとして変数を準備
    grid_x: Variable = Variable('grid_x')
    co2: Variable = Variable('co2')
    src: Variable = Variable('src')

    t: float = 0
    it: int = 0

    # アニメーションの準備
    figure, axes = plt.subplots(1, 1, figsize=(5, 5))
    results_t: list[float] = [t]
    results_co2: list[FloatArray] = [co2.value.copy()]

    while t < T_END:

        # 方程式にしたがって時間発展(Euler法)
        co2.value += (D * co2.laplacian() + src.value) * DT
        co2.set_boundary_condition(D, DT, src, C_OUT)
        t += DT

        # アニメーション用に結果を保存
        if it % int(DT_PLOT / DT) == 0:
            results_t.append(t)
            results_co2.append(co2.value.copy())
        it += 1

    # アニメーションを作成
    anim = animation.FuncAnimation(figure, create_plot, range(len(results_t)),
                                   fargs=(grid_x.value, results_t, results_co2))
    display(HTML(anim.to_jshtml()))
    plt.close()